# Sentence Similarity

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

import scipy.spatial as ss
from sentence_transformers import SentenceTransformer

from importlib.util import find_spec
if find_spec("similarity_abstract_search") is None:
    import sys
    sys.path.append('..')
    
from similarity_abstract_search.datasets.dataset import SemanticCorpusDataset
from similarity_abstract_search.models.tfidf_model import TfidfModel  
from similarity_abstract_search import utils

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
data = SemanticCorpusDataset()
data.load_one_batch()

,id,paperAbstract,title,citeEmbeddingsID
EmbeddingID,,,,
10291659,3b9b9509bb4eea4711515a40a7f210d8561487aa,Anterior segment dysgeneses are developmental ...,The 6p25 deletion syndrome: An update on a rar...,"[12279879, 11377917, 10611622, 4885766, 671395..."
13584557,c4c1d7a8051a5ee1562dc8b5722c3353b8d7e226,"The first synthesis of dolabelide C (1), a cyt...",Total synthesis of dolabelide C: a phosphate-m...,[5007102]
9811857,e300444162aa59fe05c18f99e36db8a1125275c3,BACKGROUND & AIMS: Helicobacter pylori eradica...,Effects of Community Screening for Helicobacte...,"[9936039, 1851207, 446836, 11518128, 9864285, ..."
764869,57c98b205d48d605f17d884bd6abe9a66c846989,BACKGROUND\nMinorities are more prevalent than...,Racial and ethnic disparities in physical abus...,"[6924879, 457180]"
10295489,e316c2902c421370001baf099e439cf68bef62fa,"Today, many tetraplegics benefit from surgical...",New concepts on treatment of the upper limb in...,"[7564871, 10920400, 13471692, 3174646, 10921211]"
...,...,...,...,...
2262933,640d8d35e96f819f7374c2171620782709ca96f8,Resonance frequency analysis (RFA) and Periote...,The influence of fixation in formalin on the m...,"[877477, 4908427, 3538244]"
7214837,e3eb267fbf508e3178ce0daf16941f812cb22e89,Acid ecto-phosphatase activity has been implic...,The Leishmania donovani histidine acid ecto-ph...,"[12773399, 8025331, 2428553, 9379577, 6947986,..."
3194490,a56c28c136bdf97fc6fc6e7f79240714a29de52e,Reperfusion after global ischemia (10–60 min i...,Mitochondrial Hyperoxidation Signals Residual ...,"[5510006, 813600, 3880848, 4169492, 10016922, ..."


In [29]:
df = data[0]

## Semantic Search

In [2]:
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:37<00:00, 10.8MB/s]


In [3]:
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'A cheetah is running behind its prey.'
          ]

In [4]:
corpus_embeddings = embedder.encode(corpus)

In [5]:
queries = ['A man is eating pasta.', 
    'Someone in a gorilla costume is playing a set of drums.', 
    'A cheetah chases prey on across a field.']
query_embeddings = embedder.encode(queries)

In [7]:
closest_n = 5
for query, query_embedding in zip(queries, query_embeddings):
    distances = ss.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))





Query: A man is eating pasta.

Top 5 most similar sentences in corpus:
A man is eating a piece of bread. (Score: 0.8480)
A man is eating food. (Score: 0.7759)
Two men pushed carts through the woods. (Score: 0.2095)
A monkey is playing drums. (Score: 0.1945)
A man is riding a white horse on an enclosed ground. (Score: 0.1586)




Query: Someone in a gorilla costume is playing a set of drums.

Top 5 most similar sentences in corpus:
A monkey is playing drums. (Score: 0.7985)
A cheetah is running behind its prey. (Score: 0.2860)
The girl is carrying a baby. (Score: 0.2351)
A man is riding a horse. (Score: 0.2023)
A man is riding a white horse on an enclosed ground. (Score: 0.1963)




Query: A cheetah chases prey on across a field.

Top 5 most similar sentences in corpus:
A cheetah is running behind its prey. (Score: 0.9007)
Two men pushed carts through the woods. (Score: 0.3662)
A monkey is playing drums. (Score: 0.3061)
A man is riding a horse. (Score: 0.2930)
A man is riding a whit

# TD-IDF

In [30]:
corpus = [f'{t} . {a} ' for t, a in zip(df.title, df.paperAbstract)]

In [31]:
model = TfidfModel()

In [32]:
%%time
X, V = model.fit(corpus)
X.shape

CPU times: user 22.1 ms, sys: 511 µs, total: 22.6 ms
Wall time: 22 ms


(64, 417)

In [33]:
IX=model.svmSimilarity(X)

100%|██████████| 64/64 [00:00<00:00, 528.33it/s]
